<a href="https://colab.research.google.com/github/AviTirto/TensorFlowProjects/blob/main/OptimizedCatsVsDogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extraction

Loading the data from tensorflow datasets, with 80% for training, 10% for validation, and 10% for testing

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_addons as tfa

train_data, info = tfds.load("cats_vs_dogs", split = "train[:80%]", as_supervised = True, with_info = True)
valid_data = tfds.load("cats_vs_dogs", split = "train[80%:90%]", as_supervised = True)
test_data = tfds.load("cats_vs_dogs", split = "train[90%:]", as_supervised = True)

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/4.0.1.incompleteQK11QI/cats_vs_dogs-train.tfrecord*...:   0%|…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [2]:
file_pattern = "/root/tensorflow_datasets/cats_vs_dogs/4.0.1/cats_vs_dogs-train.tfrecord*"
files = tf.data.Dataset.list_files(file_pattern)

Loading the raw TFRecord files into a dataset. The number of records processed by each thread are automatically set to the number of CPU cores(since cycle_length was not set).

The number of threads generated is automatically set based on the number of CPU cores as well.

In [3]:
train_dataset = files.interleave(
    tf.data.TFRecordDataset,
    num_parallel_calls = tf.data.experimental.AUTOTUNE # Automatically setting to number of CPU cores.
)

# Transformation

In [4]:
print(info)

tfds.core.DatasetInfo(
    name='cats_vs_dogs',
    full_name='cats_vs_dogs/4.0.1',
    description="""
    A large set of images of cats and dogs. There are 1738 corrupted images that are dropped.
    """,
    homepage='https://www.microsoft.com/en-us/download/details.aspx?id=54765',
    data_dir=PosixGPath('/tmp/tmpk09mhvsitfds'),
    file_format=tfrecord,
    download_size=786.67 MiB,
    dataset_size=1.04 GiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'image/filename': Text(shape=(), dtype=string),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=23262, num_shards=16>,
    },
    citation="""@Inproceedings (Conference){asirra-a-captcha-that-exploits-interest-aligned-manual-image-categorization,
    author = {Elson, Jeremy and Douceur, John (JD) and Howell, Jon and Saul, Jared},
    

In [5]:
def read_tfrecord(serialized_example):
  # Data Record Format
  feature_description={
    "image": tf.io.FixedLenFeature((), tf.string, ""),
    "label": tf.io.FixedLenFeature((), tf.int64, -1),
  }
  example = tf.io.parse_single_example(serialized_example, feature_description) # Transform Raw into Expected Record Format
  image = tf.io.decode_jpeg(example['image'], channels=3)  # Decodes JPEG into Image Buffer
  image = tf.cast(image, tf.float32)
  image = image / 255 # Normalization
  image = tf.image.resize(image, (300,300)) # Sets Target Shape
  image = tf.image.random_flip_left_right(image) # Random Flip(horizontally)
  image = tf.image.random_flip_up_down(image) # Random Flip(vertically)
  image = tfa.image.rotate(image, 40, interpolation = "NEAREST") # Random Rotation up to 40%
  return image, example['label']

In [6]:
import multiprocessing
cores = multiprocessing.cpu_count()
train_dataset = train_dataset.map(read_tfrecord, num_parallel_calls=cores) # The transformation was parallelized(# of threads = # of cpu cores)

In [7]:
train_dataset = train_dataset.shuffle(1024).batch(32)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

# Load

In [8]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(16, (3,3), activation = "relu", input_shape = (300, 300, 3)),
    MaxPooling2D(2,2),
    Conv2D(32, (3,3), activation = "relu"),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation = "relu"),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation = "relu"),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation = "relu"),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation = "relu"),
    Dense(1, activation = "sigmoid")
])

model.compile(
    loss = "binary_crossentropy",
    optimizer = "rmsprop",
    metrics = ['accuracy']
)

model.fit(train_dataset, epochs = 15, verbose = 1)

Epoch 1/15
727/727 [==============================] - 163s 207ms/step - loss: 0.6362 - accuracy: 0.6288
Epoch 2/15
727/727 [==============================] - 167s 218ms/step - loss: 0.5495 - accuracy: 0.7229
Epoch 3/15
727/727 [==============================] - 159s 211ms/step - loss: 0.4907 - accuracy: 0.7636
Epoch 4/15
727/727 [==============================] - 156s 208ms/step - loss: 0.4469 - accuracy: 0.7915
Epoch 5/15
727/727 [==============================] - 155s 204ms/step - loss: 0.4108 - accuracy: 0.8139
Epoch 6/15
727/727 [==============================] - 154s 205ms/step - loss: 0.3745 - accuracy: 0.8330
Epoch 7/15
727/727 [==============================] - 156s 207ms/step - loss: 0.3407 - accuracy: 0.8525
Epoch 8/15
727/727 [==============================] - 160s 212ms/step - loss: 0.3179 - accuracy: 0.8635
Epoch 9/15
727/727 [==============================] - 159s 212ms/step - loss: 0.2944 - accuracy: 0.8731
Epoch 10/15
727/727 [==============================] - 158s 209m